In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas


In [ ]:
filename = '/content/drive/Shareddrives/Reddit/month/Dec-2019-no-adult-top-10000-4-dif-subreddits.json'

In [ ]:
import pandas as pd
import json
#read data in chunks of 1 million rows at a time

with open(filename) as f:
    data = pd.DataFrame(json.loads(line) for line in f)


In [ ]:
data.head()

,ruscheinskyc,rockyp32,AmInATizzy,randomblackkiddd,GKrollin,zuccinibikini,Accountnum3billion,JohnX-ray,PsychoTexan,grandmas_funtime,victorcanfly,AutoModerator,The_One_H1T_Wonder,auamethyst,brokenarrow,elitist_ferret,CrackedOutGoose,mikebryn,Wet-Macaroni,bendstraw,nurley,archlinuxisalright,TheInternetPolice2,Immrspy,neremarine,Amargosamountain,Dorito_Dragon,GoBuffaloes,jaboi1080p,dylan2451,elkstwit,gisellasaurus,ahumanshambles,robberbaronBaby,Iorith,tujuggernaut,gtickno2,KittKattKait,BidoofTheGod,Cthulhuseye,...,IronGoddess_RS,competitive-dust,Lillianeck,captain-burrito,mdduser728,monkwillpower,daddysalad,cmrncstn1,I_Hate_Your_gutzzz,ZatoKatzke,PhilTheBiker,itsgordon,PurpleEngineer,abyymartell,TilikumBB,talkbackgetsmack,Lovenevershuns,therewasacatwizard,beyond_gone_brb,Cheesecakeius,alexvirgo13,7wouldntyouknow7,Thinkingofm,Brandonkaimusic,sad_s0ng,Shotcoplifting666,rpkittkattkitten,chugginmilk,seimc,_Pottatis,tatertot669,mrmrstaylor,Clum5y_BE,thizzy85,Ezekiel_Blufur,ruchimac6,choccy_M1Lk,TraubeOhneZucker,Ph1l1pp1us,madajurado
0,"{'Bad_Cop_No_Donut': 2, 'DontFundMe': 2, 'brit...","{'Naruto': 1, 'asktrp': 3, 'Semenretention': 2...","{'britishproblems': 9, 'JUSTNOMIL': 5, 'TheSil...","{'unpopularopinion': 27, 'atheism': 7, 'Rateme...","{'Connecticut': 7, 'BikiniBottomTwitter': 2, '...","{'wholesomememes': 2, 'psychedelicrock': 9, 'n...","{'DuelLinks': 1, 'ConsumeProduct': 14, 'stupid...","{'sweden': 1, 'ChoosingBeggars': 1, 'Technobla...","{'TankPorn': 18, 'blursedimages': 25, 'CrappyD...","{'Makeup': 4, 'russia': 1, 'astrology': 7, 'Hy...","{'Watchmen': 36, 'fo76': 5, 'Fallout': 1, 'fo7...","{'PoliticalHumor': 408, 'pokemontrades': 31108...","{'starterpacks': 2, 'Nicegirls': 3, 'gatekeepi...","{'vegan': 4, 'fuckeatingdisorders': 20, 'antiM...","{'baseball': 1, 'politics': 12, 'orlando': 1, ...","{'bestoflegaladvice': 97, 'criticalrole': 5, '...","{'fantasyfootball': 1, 'DenverBroncos': 3, 'Fo...","{'shittytechnicals': 5, 'justdependathings': 4...","{'teenagers': 109, 'AquaticAsFuck': 1, 'unpopu...","{'FIFA': 61, 'ProgrammerHumor': 1, 'StarWars':...","{'RocketLeagueEsports': 40, 'politics': 1, 'Ro...","{'changemyview': 21, 'politics': 51, 'TopMinds...","{'DankMemesFromSite19': 5, 'dogelore': 5, 'Wat...","{'Shitstatistssay': 4, 'Justrolledintotheshop'...","{'SequelMemes': 2, 'pcmasterrace': 5, 'animeme...","{'TheoryOfReddit': 9, 'HotWheels': 12, 'arrest...","{'PewdiepieSubmissions': 2, 'funkopop': 9, 'dr...","{'toptalent': 1, 'fantasyfootball': 10, 'White...","{'space': 2, 'rickandmorty': 10, 'SiliconValle...","{'ProgrammerHumor': 1, 'ShinyPokemon': 9, 'CRe...","{'editors': 3, 'MovieDetails': 1, 'Gunners': 8...","{'MakeupAddictionCanada': 2, 'AmItheAsshole': ...","{'australia': 1, 'femalefashionadvice': 1, 'sy...","{'CapitalismVSocialism': 14, 'FreeSpeech': 2, ...","{'GetMotivated': 2, 'BlackPeopleTwitter': 20, ...","{'F1Technical': 16, 'TechnoProduction': 6, 'mo...","{'Asexual': 5, 'HazbinHotel': 79, 'asexuality'...","{'BeautyBoxes': 2, 'MakeupAddiction': 1, 'MUAo...","{'fantasyfootball': 53, 'hiphopheads': 4, 'mil...","{'LeopardsAteMyFace': 10, 'boomershumor': 1, '...",...,"{'DrawForMe': 2, 'cuddleroll': 1, 'MergeDragon...","{'coolguides': 1, 'lotr': 2, 'TIHI': 1, 'marve...","{'insaneparents': 1, 'im14andthisisdeep': 1, '...","{'chinesefood': 1, 'unitedkingdom': 17, 'askga...","{'TrueOffMyChest': 7, 'amiugly': 1, 'LongDista...","{'depression': 9, 'getdisciplined': 1, 'nosurf...","{'PewdiepieSubmissions': 1, 'MovieDetails': 2,...","{'CrappyDesign': 1, 'FixedGearBicycle': 1, 'ri...","{'WatchPeopleDieInside': 41, 'Cringetopia': 1,...","{'JustBootThings': 8, 'ACQR': 3, 'battlefield_...","{'Conservative': 1, 'pittsburgh': 1, 'WhitePeo...","{'lyftdrivers': 2, 'gmod': 1, 'medical': 3, 'p...","{'AmItheAsshole': 10, '90DayFiance': 2, 'Dunde...","{'DesignMyRoom': 1, 'AmateurRoomPorn': 1, 'mal...","{'TikTokCringe': 1, 'ClimbingCircleJerk': 1, '...","{'RiseofKingdoms': 8, 'AustralianPolitics': 1,...","{'bipolar2': 2, 'exjw': 1, 'depression':

In [ ]:
new_data = data.T

In [ ]:
new_data.head()

,0
ruscheinskyc,"{'Bad_Cop_No_Donut': 2, 'DontFundMe': 2, 'brit..."
rockyp32,"{'Naruto': 1, 'asktrp': 3, 'Semenretention': 2..."
AmInATizzy,"{'britishproblems': 9, 'JUSTNOMIL': 5, 'TheSil..."
randomblackkiddd,"{'unpopularopinion': 27, 'atheism': 7, 'Rateme..."
GKrollin,"{'Connecticut': 7, 'BikiniBottomTwitter': 2, '..."


In [ ]:
n = 2000
list_df = [new_data[i:i+n] for i in range(0,new_data.shape[0],n)]
matrix_df = pd.DataFrame()
"""
n = 1000

rem = new_data.shape[0]%n
for i in range(0,new_data.shape[0],n):
  pd.json_normalize(new_data[i:i+n])
"""

'\nn = 1000\n\nrem = new_data.shape[0]%n\nfor i in range(0,new_data.shape[0],n):\n  pd.json_normalize(new_data[i:i+n])\n'

In [ ]:
for i,chunk in enumerate(list_df):
  print(str(i) + ' chunk done')
  list_df[i]=pd.json_normalize(chunk[0])
  if(i==0):
    break
  

0 chunk done


In [ ]:
#list_df[0].to_csv (r'/content/drive/Shareddrives/Reddit/batch0.csv', index = True, header=True)

In [ ]:
matrix_df = list_df[0]

In [ ]:
print(matrix_df.head)

<bound method NDFrame.head of       Bad_Cop_No_Donut  DontFundMe  ...  Hatfilms  chemhelp
0                  2.0         2.0  ...       NaN       NaN
1                  NaN         NaN  ...       NaN       NaN
2                  NaN         NaN  ...       NaN       NaN
3                  NaN         NaN  ...       NaN       NaN
4                  NaN         NaN  ...       NaN       NaN
...                ...         ...  ...       ...       ...
1995               NaN         NaN  ...       NaN       NaN
1996               NaN         NaN  ...       NaN       NaN
1997               NaN         NaN  ...       NaN       NaN
1998               NaN         NaN  ...       NaN       NaN
1999               NaN         NaN  ...       NaN       NaN

[2000 rows x 5731 columns]>


In [ ]:
import numpy as np

train_index = np.random.random(len(matrix_df)) <= 0.7
train_df = matrix_df[train_index]
test_df = matrix_df[~train_index]

In [ ]:
print(train_df.head())

print(test_df.head())

   Bad_Cop_No_Donut  DontFundMe  ...  Hatfilms  chemhelp
3               NaN         NaN  ...       NaN       NaN
6               NaN         NaN  ...       NaN       NaN
7               NaN         NaN  ...       NaN       NaN
8               NaN         NaN  ...       NaN       NaN
9               NaN         NaN  ...       NaN       NaN

[5 rows x 5731 columns]
   Bad_Cop_No_Donut  DontFundMe  ...  Hatfilms  chemhelp
0               2.0         2.0  ...       NaN       NaN
1               NaN         NaN  ...       NaN       NaN
2               NaN         NaN  ...       NaN       NaN
4               NaN         NaN  ...       NaN       NaN
5               NaN         NaN  ...       NaN       NaN

[5 rows x 5731 columns]


In [ ]:
train_df.fillna(0)
subreddits = train_df.columns
print(subreddits)
train_mat = train_df.to_numpy()
print(train_mat)

Index(['Bad_Cop_No_Donut', 'DontFundMe', 'britishproblems', 'PrequelMemes',
       'politics', 'quityourbullshit', 'Cringetopia', 'freefolk', 'trashy',
       'comedynecromancy',
       ...
       '32dollars', 'catswhotrill', 'PointlessStories', 'Homesteading',
       'youtubetv', 'badassanimals', 'DamnNatureYouScary', 'whatcarshouldIbuy',
       'Hatfilms', 'chemhelp'],
      dtype='object', length=5731)
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


In [ ]:
prediction_mat = np.ones_like(train_mat)


In [ ]:

user_train_like = []
for u in range(len(train_mat)):
    user_train_like.append(np.where(train_mat[u, :] > 0)[0])

numer = np.matmul(train_mat, train_mat.T)
denom = np.sum(train_mat ** 2, axis=1, keepdims=True) ** 0.5
Cosine = numer / np.matmul(denom, denom.T)

recommendation = []
for u in range(len(train_mat)):
    similarities = Cosine[u, :]
    similarities[u] = -1
    N_idx = np.argpartition(similarities, -5)[-5:]
    N_sim = similarities[N_idx]
    scores = np.sum(N_sim.reshape((-1, 1)) * train_mat[N_idx, :], axis=0) / np.sum(N_sim)
    
    train_like = user_train_like[u]
    scores[train_like] = -9999
    top50_iid = np.argpartition(scores, -5)[-5:]
    top50_iid = top50_iid[np.argsort(scores[top50_iid])[-1::-1]]
    recommendation.append(top50_iid)
recommendation = np.array(recommendation)

In [ ]:
print(recommendation)

[[5730 1914 1915 1913 1911]
 [5730 1915 1916 1914 1912]
 [5730 1915 1916 1914 1912]
 ...
 [5730 1912 1913 1911 1909]
 [5730 1912 1913 1911 1909]
 [5730 1914 1915 1913 1911]]


In [ ]:
#Comments made by the user in question
print(new_data[0][1016])
#Recommendations by subreddit ID
print(recommendation[1016], '\n')

#Prints recommended subreddits by name
for i in recommendation[1016]:
 print(subreddits[i])

{'Random_Acts_Of_Amazon': 27, 'SantasLittleHelpers': 14, 'relationships': 1, 'MakeupAddiction': 3, 'Anticonsumption': 1, 'intermittentfasting': 1, 'happy': 4, 'relationship_advice': 3, 'Assistance': 6, 'Entrepreneur': 1, 'CasualConversation': 1, 'NarcissisticAbuse': 3, 'raisedbyborderlines': 1, 'ZeroWaste': 1, 'addiction': 1, 'Visiblemending': 1, 'RandomActsofCards': 2}
[5730 1919 1920 1918 1916] 

chemhelp
MakeupRehab
Lyft
Boise
homeautomation
